<span STYLE="font-size:150%"> 
    Calculate concentrations from emission data
</span>

Docker image: gnasello/datascience-env:2023-03-30 \
Latest update: 31 March 2023

# Load libraries

In [ ]:
library(ggplot2)
library(latex2exp)
library(ggpubr)
library(yaml)
library(dplyr)

Clone the [r_utils library](https://github.com/gabnasello/r_utils.git) from GitHub in the same folder of this script. 

You can simply running the following command in a new terminal (open it from JupyterLab):

`git clone https://github.com/gabnasello/r_utils.git`

How to Reuse Functions That You Create In Scripts, [tutorial](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-data/source-function-in-R/)

In [ ]:
source("r_utils/ggplot_utils.R")

# Reading data

<span style="color:red">**User Input**</span>

In [ ]:
filename <- "data/2023-03-28_GN011.csv"

In [ ]:
# Import the data and look at the first six rows
df <- read.csv(file = filename)
head(df)

# Select standard samples

In [ ]:
prefix <- 'SPL'

In [ ]:
condition <- startsWith(df$Well.ID, prefix)

df_spl <- df[condition,]
head(df_spl)

# Calculate concentrations

Determine sample dilution

<span style="color:red">**User Input**</span>

In [ ]:
dilution = 4

## Load regression model of the calibration curve

In [ ]:
model <- readRDS("calibration.rds")
model

## Compute concentrations based on calibration curve

Of the samples

In [ ]:
sampledata <- df_spl %>% select(Mean_blnk)

df_spl$concentration <- predict(model, sampledata) * dilution
head(df_spl)

Of the whole dataset

In [ ]:
newdata <- df %>% select(Mean_blnk)

df$concentration <- predict(model, newdata) * dilution
head(df)

## Save data

Overwrite initial dataset after calculating the sample concentration

In [ ]:
write.csv(df, filename, row.names=FALSE)

# Calculate amount (percentage) released

## Get the volume of the released media for each sample tested

Prepare and load the `sample_info.csv` file

In [ ]:
info_file <- "data/samples_info.csv"

In [ ]:
# Import the data and look at the first six rows
df_info <- read.csv(file = info_file)
head(df_info)

Merge `df` and `df_info` using the sample ID info and droppoing the not sample data in `df` 

In [ ]:
df_release <- merge(df_spl, df_info, by = "Well.ID", 
                   all.x = FALSE, all.y = FALSE, sort=FALSE)
head (df_release)

Alternatively, you can select a volume value equal for all samples by uncommenting the cell below

In [ ]:
# volume_sample <- 0.450 #mul
# df$volume <- volume_sample

## Compute amounts (percentage) released

<span style="color:red">**User Input**</span>

In [ ]:
drug_amount = 200 #ug

In [ ]:
df_release$release = df_release$concentration * df_release$volume / drug_amount * 100
head(df_release)

## Save data

Overwrite initial dataset after calculating the amount of drug released at each timepoint

In [ ]:
new_filename <- paste(tools::file_path_sans_ext(filename), '_release.csv', sep='')
new_filename

In [ ]:
write.csv(df_release, new_filename, row.names=FALSE)

# Info on units

If the concentration of the standard samples used to build the calibration curve is in *ug/ml*, then:

1. `df$concentration` is also in  ug/ml
2. `df$volume` must be in ml
3. `df$release` is in % of the total drug loaded into the gel